In [6]:
# 1. Install TensorFlow 2.19.0
!pip install -q tensorflow==2.19.0  # Instal versi TensorFlow 2.19.0 yang kompatibel

# 2. Upgrade ml-dtypes ke versi yang sesuai dengan TensorFlow 2.19.0
!pip install -q ml-dtypes==0.5.1  # Sesuaikan dengan kebutuhan TensorFlow

# 3. Install paket lain yang diperlukan, pastikan versinya cocok
!pip install -q kaggle opencv-python-headless matplotlib tensorflow-text==2.19.0 jaxlib==0.7.2 tensorstore==0.1.78


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 69.8 MB/s eta 0:00:00


In [7]:
import tensorflow as tf
import ml_dtypes

print("TensorFlow version:", tf.__version__)
print("ml-dtypes version:", ml_dtypes.__version__)


TensorFlow version: 2.19.0
ml-dtypes version: 0.5.1


In [8]:
# 2. Upload kaggle.json (API token) ke Colab:
# - pergi ke https://www.kaggle.com -> Account -> Create API Token -> upload kaggle.json
from google.colab import files
print("Upload kaggle.json (dari akun Kaggle Anda)")
files.upload()  # upload kaggle.json di sini


Upload kaggle.json (dari akun Kaggle Anda)


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"farhanfaturrachman","key":"a0f132627190b5a8ba71525f562b68b0"}'}

In [9]:
# 3. Atur kredensial dan download dataset MRL
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ganti 'prasadvpatil/mrl-dataset' bila berbeda
!kaggle datasets download -d prasadvpatil/mrl-dataset -p /content/data --unzip
!ls -lah /content/data


Dataset URL: https://www.kaggle.com/datasets/prasadvpatil/mrl-dataset
License(s): CC0-1.0
  0% 0.00/22.6M [00:00<?, ?B/s]
100% 22.6M/22.6M [00:00<00:00, 1.45GB/s]
total 12K
drwxr-xr-x 3 root root 4.0K Nov  1 05:49 .
drwxr-xr-x 1 root root 4.0K Nov  1 05:49 ..
drwxr-xr-x 4 root root 4.0K Nov  1 05:49 train


In [10]:
# 4. Inspect folder structure -> cari folder label (Open/Closed atau serupa)
import os
base = "/content/data"
for root, dirs, files in os.walk(base):
    # print top-level only
    print(root, len(dirs), "dirs", len(files), "files")
    break

# jika data ada dalam subfolder, sesuaikan data_dir di bawah
data_dir = "/content/data/MRL_Dataset"  # *sesuaikan* bila struktur berbeda


/content/data 1 dirs 0 files


In [11]:
# 5. Jika dataset berisi subfolder per kelas (Open_Eyes, Closed_Eyes), kita bisa pakai ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib

# coba auto-detect kelas dari folder
def find_data_dir_guess():
    # cari folder yang mengandung 'Open' atau 'Closed' atau 'open_eyes' dll
    for p in pathlib.Path("/content/data").rglob("*"):
        if p.is_dir():
            # check if contains typical class folders
            names = [q.name.lower() for q in p.iterdir() if q.is_dir()]
            if any('open' in n or 'close' in n or 'closed' in n for n in names):
                return str(p)
    return "/content/data"

data_dir = find_data_dir_guess()
print("Menggunakan data_dir =", data_dir)
print("Isi:", os.listdir(data_dir)[:20])


Menggunakan data_dir = /content/data/train
Isi: ['Closed_Eyes', 'Open_Eyes']


In [12]:
# 6. Parameters
IMG_SIZE = (96,96)          # ukuran input model (ubah jika mau)
BATCH_SIZE = 32
EPOCHS = 20


In [13]:
# 7. Data generators (train/val split)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.08,
    height_shift_range=0.08,
    shear_range=0.05,
    zoom_range=0.08,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='binary',   # binary => open vs closed
    subset='training',
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# cek class indices
print("Classes:", train_gen.class_indices)


Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Classes: {'Closed_Eyes': 0, 'Open_Eyes': 1}


In [14]:
# 8. Build a simple but effective CNN
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = IMG_SIZE + (3,)

model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 96, 96, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 96, 96, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,359,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,454,209 (9.36 MB)

 Trainable params: 2,453,505 (9.36 MB)

 Non-trainable params: 704 (2.75 KB)

In [15]:
# 9. Train with callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, es, rlr]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.8747 - loss: 0.2793

100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.8752 - loss: 0.2782 - val_accuracy: 0.5000 - val_loss: 1.5502 - learning_rate: 1.0000e-04
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 127ms/step - accuracy: 0.9808 - loss: 0.0602 - val_accuracy: 0.5000 - val_loss: 1.8016 - learning_rate: 1.0000e-04
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 0.9873 - loss: 0.0418 - val_accuracy: 0.5000 - val_loss: 1.6293 - learning_rate: 1.0000e-04
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9938 - loss: 0.0253

100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.9938 - loss: 0.0253 - val_accuracy: 0.6787 - val_loss: 0.7303 - learning_rate: 1.0000e-04
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9932 - loss: 0.0204

100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9932 - loss: 0.0204 - val_accuracy: 0.9475 - val_loss: 0.1456 - learning_rate: 1.0000e-04
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.9953 - loss: 0.0218

100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9952 - loss: 0.0218 - val_accuracy: 0.9875 - val_loss: 0.0629 - learning_rate: 1.0000e-04
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9910 - loss: 0.0241 - val_accuracy: 0.9675 - val_loss: 0.0846 - learning_rate: 1.0000e-04
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.9941 - loss: 0.0203 - val_accuracy: 0.9550 - val_loss: 0.1140 - learning_rate: 1.0000e-04
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.9967 - loss: 0.0104 - val_accuracy: 0.9825 - val_loss: 0.0495 - learning_rate: 1.0000e-04
Epoch 10/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.9964 - loss: 0.0109 - val_accuracy: 0.9688 - val_loss: 0.0786 - learning_rate: 1.0000e-04
Epoch 11/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 116ms/step - accuracy: 0.9994 - loss: 0.0055 - val_accuracy: 0.9862 - val_loss: 0.0415 - learning_rate: 1.0000e-04
Epoch 12/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step

100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9979 - loss: 0.0087 - val_accuracy: 0.9950 - val_loss: 0.0318 - learning_rate: 1.0000e-04
Epoch 14/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9991 - loss: 0.0048 - val_accuracy: 0.9875 - val_loss: 0.0410 - learning_rate: 1.0000e-04
Epoch 15/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9984 - loss: 0.0071 - val_accuracy: 0.9625 - val_loss: 0.0939 - learning_rate: 1.0000e-04
Epoch 16/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9959 - loss: 0.0109 - val_accuracy: 0.9812 - val_loss: 0.0557 - learning_rate: 1.0000e-04
Epoch 17/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 0.9976 - loss: 0.0081 - val_accuracy: 0.9837 - val_loss: 0.0504 - learning_rate: 5.0000e-05
Epoch 18/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9997 - loss: 0.0044 - val_accuracy: 0.9775 - val_loss: 0.0567 - learning_rate: 5.0000e-05
Epoch 19/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/s

In [16]:
# 10. Evaluate on validation set
model.load_weights("best_model.h5")
loss, acc = model.evaluate(val_gen)
print("Val loss:", loss, "Val acc:", acc)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.9886 - loss: 0.0281
Val loss: 0.0552511066198349 Val acc: 0.9775000214576721


In [17]:
# 11. Save the Keras model (h5) and convert to TFLite
model.save("drowsiness_model.h5")


In [21]:
# Convert to TFLite (float32) first
import tensorflow as tf

keras_model = tf.keras.models.load_model("drowsiness_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()
open("drowsiness_model.tflite", "wb").write(tflite_model)
print("Saved drowsiness_model.tflite (float32)")

# Convert to TFLite (dynamic range quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic_quant = converter.convert()
open("drowsiness_model_dynamic_quant.tflite", "wb").write(tflite_model_dynamic_quant)
print("Saved drowsiness_model_dynamic_quant.tflite (dynamic range quantized)")

# Convert to TFLite (float16 quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model_fp16 = converter.convert()
open("drowsiness_model_fp16.tflite", "wb").write(tflite_model_fp16)
print("Saved drowsiness_model_fp16.tflite (float16 quantized)")

Saved artifact at '/tmp/tmp5ewq3bp5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135729289938512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289951952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289937552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289950416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289938896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289951760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289951568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289951184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289950800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289938320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135729289951376: 

In [22]:
# 12. Test inference with TFLite on a sample image
import numpy as np
from PIL import Image

interpreter = tf.lite.Interpreter(model_path="drowsiness_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input details:", input_details)
print("Output details:", output_details)

# pick an example image from val_gen
x_batch, y_batch = next(val_gen)
img = x_batch[0]
# prepare input (already scaled to 0-1 by generator)
inp = np.expand_dims(img.astype(np.float32), axis=0)
interpreter.set_tensor(input_details[0]['index'], inp)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0]['index'])
print("Pred raw:", pred, "label:", y_batch[0])


Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([ 1, 96, 96,  3], dtype=int32), 'shape_signature': array([-1, 96, 96,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 33, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Pred raw: [[0.0004684]] label: 0.0


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [23]:
# 13. Download the TFLite files to local
from google.colab import files
files.download("drowsiness_model.tflite")
files.download("drowsiness_model_dynamic_quant.tflite")
files.download("drowsiness_model_fp16.tflite")
files.download("drowsiness_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>